# 非线性
***

下一个示例主要展示了如何解决非线性系统方程调用函数的问题。

首先我们以一个简单的模型开始介绍。该模型包含了非常简单的数学关系，如下所示：

$$y=2t^2+3t+1$$

其中，$t$表示时间。上述模型在Modelica中的实现如下：


```Modelica
model ExplicitEvaluation
  "Model that evaluates the quadratic function explicitly"
  Real y;
equation
  y = Quadratic(2.0, 3.0, 1.0, time);
end ExplicitEvaluation;
```

其中，Quadratic函数用于二次多项式的计算。我们在后面会对其进行简单的讨论。

对上述模型进行仿真，变量y的仿真结果如下图所示：


<img src="./figure4\7.png" width="500"  height="400" alt="二次多项式的计算" align= center />


目前为止，所有的一切似乎都很合理。基于前面章节多项式计算中的讨论，Quadratic函数的使用似乎也在意料之中。但是，对于如下所示的更加复杂的模型：


```Modelica
model ImplicitEvaluation
  "Model that requires the inverse of the quadratic function"
  parameter Real y_guess=2;
  Real y(start=y_guess);
equation
  time+1 = Quadratic(2.0, 3.0, 1.0, y);
end ImplicitEvaluation;
```


上述模型相当于求解下列方程：

$$t+5=2y^2+3y+1$$


明显的区别在于，上述模型中等式左边是已知的。我们需要计算满足方程的$y$值。换言之，不同于前面示例中计算二次多项式的值，上述模型需对二次多项式进行求解。


需要求解非线性系统方程的模型并不太好。Modelica编译器具有非凡识别和求解非线性系统方程的能力（尽管为了收敛，这些方法通常需依赖于合理的初始猜测）。


但事实上，上述情况下Modelica编译器并不需要求解一个非线性系统。若我们了解了Quadratic函数是如何实现的，就能清楚的了解上述情况：


```Modelica
function Quadratic "A quadratic function"
  input Real a "2nd order coefficient";
  input Real b "1st order coefficient";
  input Real c "constant term";
  input Real x "independent variable";
  output Real y "dependent variable";
algorithm
  y := a*x*x + b*x + c;
  annotation(inverse(x = InverseQuadratic(a,b,c,y)));
end Quadratic;
```

特别的，需重点注意上述代码中的inverse注释。对于上述函数的定义，不仅告诉Modelica编译器如何求解Quadratic方程，而且通过inverse注释利用InverseQuadratic函数可以计算得到变量x和变量y的关系。

InverseQuadratic函数的定义如下所示：


```Modelica
function InverseQuadratic
  "An inverse of the quadratic function returning the positive root"
  input Real a;
  input Real b;
  input Real c;
  input Real y;
  output Real x;
algorithm
  x := sqrt(b*b - 4*a*(c - y))/(2*a);
end InverseQuadratic;
```

>## Note
>通过上述代码，可以看到InverseQuadratic函数只能计算二次方程的正根。对我们来说，这个结果可好可坏。只计算一个根的话，在转换二次函数关系时就可以避免多个解。但是，如果你恰巧需要负根，就可能会产生相应的问题。


在上述ImplicitEvaluation模型中，Modelica编译器可以替代上述反函数方程。因此，忽略当前方程的参数，需要求解的方程如下所示：

$$t+5=f(y)$$


其中，必须求解变量$y$的隐式方程，可以得到如下所示显式方程：

$$y=f^{-1}(t+5)$$

而通过使用InverseQuadratic函数作为原函数的反函数。

对上述ImplicitEvaluation模型进行，可以得到变量y仿真结果，如下图所示：


<img src="./figure4\8.png" width="500"  height="400" alt="二次多项式的计算" align= center />


通过观察上图，可以发现，模型计算得到了正确的结果。而且，一般情况下并不需要求解非线性系统。否则，如果没有注解，那ImplicitEvaluation模型就会引入非线性的方程组。